In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging

import pandas as pd

import helpers.hdbg as hdbg
import optimizer.single_period_optimization as osipeopt

In [3]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

# _LOG.info("%s", henv.get_system_signature()[0])

INFO: > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-ac6c4b23-195a-42de-a1e3-12df46319ebc.json'


In [4]:
# hprint.config_notebook()

# Build forecast dataframe

In [5]:
df = pd.DataFrame(
    [
        [1, 1000, 1, 1000, 0.05, 0.05],
        [2, 1500, 1, 1500, 0.09, 0.07],
        [3, -500, 1, -500, 0.03, 0.08],
    ],
    range(0, 3),
    [
        "asset_id",
        "holdings_shares",
        "price",
        "holdings_notional",
        "prediction",
        "volatility",
    ],
)

# Build optimizer config

In [6]:
dict_ = {
    "dollar_neutrality_penalty": 0.0,
    "volatility_penalty": 0.0,
    "relative_holding_penalty": 0.0,
    "relative_holding_max_frac_of_gmv": 0.6,
    "target_gmv": 3000,
    "target_gmv_upper_bound_penalty": 0.0,
    "target_gmv_hard_upper_bound_multiple": 1.00,
    "turnover_penalty": 0.0,
    "solver": "ECOS",
}

# Optimize

In [7]:
spo = osipeopt.SinglePeriodOptimizer(dict_, df)

In [8]:
opt_results = spo.optimize()
display(opt_results.round(3))

INFO  `optimal_value`=0.22


,target_holdings_notional,target_trades_notional,target_weight,target_weight_diff
asset_id,,,,
1,1200.0,200.0,1.2,0.2
2,1800.0,300.0,1.8,0.3
3,0.0,500.0,0.0,0.5


In [9]:
res1 = opt_results.stack().rename("s1")

In [10]:
res2 = opt_results.stack().rename("s2")

In [11]:
pd.concat([res1, res2], keys=["s1", "s2"], axis=1)

s1            s2
asset_id                                                     
1        target_holdings_notional  1.200000e+03  1.200000e+03
         target_trades_notional    2.000000e+02  2.000000e+02
         target_weight             1.200000e+00  1.200000e+00
         target_weight_diff        2.000000e-01  2.000000e-01
2        target_holdings_notional  1.800000e+03  1.800000e+03
         target_trades_notional    3.000000e+02  3.000000e+02
         target_weight             1.800000e+00  1.800000e+00
         target_weight_diff        3.000000e-01  3.000000e-01
3        target_holdings_notional  3.520370e-06  3.520370e-06
         target_trades_notional    5.000000e+02  5.000000e+02
         target_weight             3.520370e-09  3.520370e-09
         target_weight_diff        5.000000e-01  5.000000e-01

In [12]:
pd.concat(
    {
        "s1": res1,
        "s2": res2,
    },
    axis=1,
).T

asset_id                        1                                       \
         target_holdings_notional target_trades_notional target_weight   
s1                    1199.999997             199.999997           1.2   
s2                    1199.999997             199.999997           1.2   

asset_id                                           2                         \
         target_weight_diff target_holdings_notional target_trades_notional   
s1                      0.2              1799.999999             299.999999   
s2                      0.2              1799.999999             299.999999   

asset_id                                                         3  \
         target_weight target_weight_diff target_holdings_notional   
s1                 1.8                0.3                 0.000004   
s2                 1.8                0.3                 0.000004   

asset_id                                                          
         target_trades_notional target_weight target_weight_diff  
s1                   500.000004  3.520370e-09                0.5  
s2                   500.000004  3.520370e-09                0.5

In [13]:
spo.compute_stats(opt_results)

,notional,percentage
gross_volume,999.999999,33.333333
net_volume,999.999999,33.333333
gmv,2999.999999,100.000000
nmv,2999.999999,100.000000


# Process forecast dataframe

In [14]:
tz = "America/New_York"
idx = [
    pd.Timestamp("2022-01-03 09:35:00", tz=tz),
    pd.Timestamp("2022-01-03 09:40:00", tz=tz),
    pd.Timestamp("2022-01-03 09:45:00", tz=tz),
    pd.Timestamp("2022-01-03 09:50:00", tz=tz),
]
asset_ids = [100, 200]

prediction_data = [
    [-0.25, -0.34],
    [0.13, 0.5],
    [0.84, -0.97],
    [0.86, -0.113],
]

price_data = [[100.0, 100.3], [100.1, 100.5], [100.05, 100.4], [100.2, 100.5]]

volatility_data = [
    [0.00110, 0.00048],
    [0.00091, 0.00046],
    [0.00086, 0.00060],
    [0.00071, 0.00068],
]

prediction_df = pd.DataFrame(prediction_data, idx, asset_ids)
price_df = pd.DataFrame(price_data, idx, asset_ids)
volatility_df = pd.DataFrame(volatility_data, idx, asset_ids)
dag_df = pd.concat(
    {"price": price_df, "volatility": volatility_df, "prediction": prediction_df},
    axis=1,
)

In [15]:
dag_df

price        volatility          prediction       
                              100    200        100      200        100    200
2022-01-03 09:35:00-05:00  100.00  100.3    0.00110  0.00048      -0.25 -0.340
2022-01-03 09:40:00-05:00  100.10  100.5    0.00091  0.00046       0.13  0.500
2022-01-03 09:45:00-05:00  100.05  100.4    0.00086  0.00060       0.84 -0.970
2022-01-03 09:50:00-05:00  100.20  100.5    0.00071  0.00068       0.86 -0.113

In [16]:
import optimizer.forecast_evaluator_with_optimizer as ofevwiop

/app/amp/optimizer/forecast_evaluator_with_optimizer.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [17]:
fewo = ofevwiop.ForecastEvaluatorWithOptimizer(
    "price",
    "volatility",
    "prediction",
    dict_,
)

In [18]:
dag_df

price        volatility          prediction       
                              100    200        100      200        100    200
2022-01-03 09:35:00-05:00  100.00  100.3    0.00110  0.00048      -0.25 -0.340
2022-01-03 09:40:00-05:00  100.10  100.5    0.00091  0.00046       0.13  0.500
2022-01-03 09:45:00-05:00  100.05  100.4    0.00086  0.00060       0.84 -0.970
2022-01-03 09:50:00-05:00  100.20  100.5    0.00071  0.00068       0.86 -0.113

In [19]:
portfolio_df, portfolio_stats_df = fewo.annotate_forecasts(
    dag_df, quantization="nearest_share"
)

  0%|          | 0/4 [00:00<?, ?it/s]

INFO  timestamp=2022-01-03 09:35:00-05:00, next_timestamp=2022-01-03 09:40:00-05:00
INFO  `optimal_value`=0.61
INFO  timestamp=2022-01-03 09:40:00-05:00, next_timestamp=2022-01-03 09:45:00-05:00
INFO  `optimal_value`=0.70
INFO  timestamp=2022-01-03 09:45:00-05:00, next_timestamp=2022-01-03 09:50:00-05:00
INFO  timestamp=2022-01-03 09:50:00-05:00, next_timestamp=None
INFO  `optimal_value`=1.12


In [20]:
portfolio_df

price        volatility          prediction  \
                              100    200        100      200        100   
2022-01-03 09:35:00-05:00  100.00  100.3    0.00110  0.00048      -0.25   
2022-01-03 09:40:00-05:00  100.10  100.5    0.00091  0.00046       0.13   
2022-01-03 09:45:00-05:00  100.05  100.4    0.00086  0.00060       0.84   
2022-01-03 09:50:00-05:00  100.20  100.5    0.00071  0.00068       0.86   

                                 holdings_shares       holdings_notional  \
                             200             100   200               100   
2022-01-03 09:35:00-05:00 -0.340             0.0   0.0               0.0   
2022-01-03 09:40:00-05:00  0.500           -12.0 -18.0           -1201.2   
2022-01-03 09:45:00-05:00 -0.970            12.0  18.0            1200.6   
2022-01-03 09:50:00-05:00 -0.113             0.0   0.0               0.0   

                                  executed_trades_shares        \
                              200                    100   200   
2022-01-03 09:35:00-05:00     0.0                    0.0   0.0   
2022-01-03 09:40:00-05:00 -1809.0                  -12.0 -18.0   
2022-01-03 09:45:00-05:00  1807.2                   24.0  36.0   
2022-01-03 09:50:00-05:00     0.0                  -12.0 -18.0   

                          executed_trades_notional          pnl       
                                               100     200  100  200  
2022-01-03 09:35:00-05:00                      0.0     0.0  0.0  0.0  
2022-01-03 09:40:00-05:00                  -1201.2 -1809.0  0.0  0.0  
2022-01-03 09:45:00-05:00                   2401.2  3614.4  0.6  1.8  
2022-01-03 09:50:00-05:00                  -1202.4 -1809.0  1.8  1.8

In [21]:
portfolio_stats_df

,pnl,gross_volume,net_volume,gmv,nmv
2022-01-03 09:35:00-05:00,0.0,0.0,0.0,0.0,0.0
2022-01-03 09:40:00-05:00,0.0,3010.2,-3010.2,3010.2,-3010.2
2022-01-03 09:45:00-05:00,2.4,6015.6,6015.6,3007.8,3007.8
2022-01-03 09:50:00-05:00,3.6,3011.4,-3011.4,0.0,0.0


In [22]:
dfs = fewo.compute_portfolio(dag_df, quantization="nearest_share")

  0%|          | 0/4 [00:00<?, ?it/s]

INFO  timestamp=2022-01-03 09:35:00-05:00, next_timestamp=2022-01-03 09:40:00-05:00
INFO  `optimal_value`=0.61
INFO  timestamp=2022-01-03 09:40:00-05:00, next_timestamp=2022-01-03 09:45:00-05:00
INFO  `optimal_value`=0.70
INFO  timestamp=2022-01-03 09:45:00-05:00, next_timestamp=2022-01-03 09:50:00-05:00
INFO  timestamp=2022-01-03 09:50:00-05:00, next_timestamp=None
INFO  `optimal_value`=1.12
